# Spotify Sentiment Analysis with Spacy and Spacytextblob

### Beth Harvey

The goal of this project is to use the Spotify API and Spotipy to get the titles of the top 5 songs from Spotify's Top Songs Global playlist, along with several Top Songs playlists for various countries where English is the primary language (only the English NLP package is used). The LyricsGenius API will be used to get the lyrics for each track, then sentiment analysis will be performend on the data using spaCy. Song data will be saved in JSON files, then sentiment analysis will be done using spacy and spacy text blob. 

The playlists are updated weekly and were accessed on August 4-6, 2023.

NOTE: Several of the songs in these playlists contain explicit lyrics. Proceed with caution.


Installations
* pip install -U pip setuptools wheel
* pip install -U spacy
* python -m spacy download en_core_web_sm
* pip install spotipy --upgrade
* pip install lyricsgenius
* pip install python-dotenv

In [1]:
# Import and confirm necessary packages
import json

import requests
import spacy
import os
from spacytextblob.spacytextblob import SpacyTextBlob
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from dotenv import load_dotenv

# print('All prereqs installed.')
# !pip list

## API Credentials
Following the Spotipy instructions at https://spotipy.readthedocs.io/en/2.22.1/ I saved my Spotify client ID, client secret, and redirect URI as environment variables. They need to be loaded in as variables so they are available for use.

In [2]:
# Read in environment variables
load_dotenv()
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect = os.getenv('SPOTIPY_REDIRECT_URI')
genius_token = os.getenv('LYRICS_GENIUS_TOKEN')

## Define Functions

### Get Top Five Tracks From Spotify Playlist Using Spotipy

In [3]:
# Define function to get a list of the top 5 songs from a playlist
def get_top_five(playlist_uri):
    # Initialize Spotipy object
    spotify = spotipy.Spotify(auth_manager = SpotifyClientCredentials())
    # Get details of top 5 songs from playlist
    top_5_songs = spotify.playlist_tracks(playlist_uri, limit = 5)
    # Get main artists and titles of top 5 tracks
    top_5_tracks = []
    full_names = []
    for track in top_5_songs['items']:
        track_artist= track['track']['artists'][0]['name']
        full_track_name = track['track']['name']
        full_names.append(full_track_name)
            # Clean track names (better search results)
        for name in full_names:
            split_1 = name.split(' (')
            split_2 = split_1[0].split(' [')
            track_name = split_2[0]

        track_info = (track_artist, track_name)
        top_5_tracks.append(track_info)
    return top_5_tracks

### Get Song Lyrics From LyricsGenius

In [4]:
# Define function to get lyrics of a song and save as a JSON file
def get_song_lyrics(track_name, track_artist):
    # Initialize LyricsGenius object
    genius = lyricsgenius.Genius(genius_token)
    song_details = genius.search_song(title = track_name, artist = track_artist)
    song_lyrics = song_details.lyrics
    file_name = f'{track_name}.json'
    # Check to see if file already exists before creating new one (some songs are in multiple playlists)
    if not os.path.exists(file_name):
        with open(file_name, 'w') as file:
            json.dump(song_lyrics, file)

### Get Polarity Score For Lyrics Using spaCy

In [5]:
def get_polarity_score(track_name):
    file_name = f'{track_name}.json'
    # Load lyrics from JSON file
    with open(file_name, 'r') as file:
        song_lyrics = json.load(file)
    # Initialize spaCy object
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    # Apply sentiment analysis to lyrics
    sentiment_analysis = nlp(song_lyrics)
    # Get polarity score
    polarity_score = sentiment_analysis._.blob.polarity
    return polarity_score

## Global Top Songs Playlist

### Get Track Names

In [6]:
# Get names of top 5 tracks from Global Top Songs
top_five_global = get_top_five('37i9dQZEVXbNG2KDcFcKOF')
print(top_five_global)

[('Jung Kook', 'Seven'), ('Myke Towers', 'LALA'), ('Travis Scott', 'MELTDOWN'), ('Billie Eilish', 'What Was I Made For?'), ('Travis Scott', 'FE!N')]


### Get Lyrics

In [7]:
# Save lyrics for global top 5 songs
for artist, song in top_five_global:
    get_song_lyrics(song, artist)

Searching for "Seven" by Jung Kook...
Done.
Searching for "LALA" by Myke Towers...
Done.
Searching for "MELTDOWN" by Travis Scott...
Done.
Searching for "What Was I Made For?" by Billie Eilish...
Done.
Searching for "FE!N" by Travis Scott...
Done.


### Sentiment Analysis

In [8]:
# Get polarity scores for Global Top Songs
top_five_global_polarity = [get_polarity_score(song) for artist, song in top_five_global]
print(top_five_global_polarity)

[0.15595238095238093, 0.11666666666666665, -0.16178026834276832, 0.09930555555555556, -0.08304473304473305]


## Top Songs USA

### Get Track Names

In [9]:
top_five_usa = get_top_five('37i9dQZEVXbLp5XoPON0wI')
print(top_five_usa)

[('Travis Scott', 'MELTDOWN'), ('Travis Scott', 'FE!N'), ('Travis Scott', 'I KNOW ?'), ('Travis Scott', 'HYAENA'), ('Gunna', 'fukumean')]


### Get Lyrics

In [10]:
for artist, song in top_five_usa:
    get_song_lyrics(song, artist)

Searching for "MELTDOWN" by Travis Scott...
Done.
Searching for "FE!N" by Travis Scott...
Done.
Searching for "I KNOW ?" by Travis Scott...
Done.
Searching for "HYAENA" by Travis Scott...
Done.
Searching for "fukumean" by Gunna...
Done.


### Sentiment Analysis

In [11]:
top_five_usa_polarity = [get_polarity_score(song) for artist, song in top_five_usa]
print(top_five_usa_polarity)

[-0.16178026834276832, -0.08304473304473305, 0.0040294976341487985, 0.10182884748102138, -0.000502953119232195]


## Top Songs UK

### Get Track Names

In [12]:
top_five_uk = get_top_five('37i9dQZEVXbMwmF30ppw50')
print(top_five_uk)

[('Dave', 'Sprinter'), ('Billie Eilish', 'What Was I Made For?'), ('Dua Lipa', 'Dance The Night'), ('Olivia Rodrigo', 'vampire'), ('Nicki Minaj', 'Barbie World')]


### Get Lyrics

In [13]:
for artist, song in top_five_uk:
    get_song_lyrics(song, artist)

Searching for "Sprinter" by Dave...
Done.
Searching for "What Was I Made For?" by Billie Eilish...
Done.
Searching for "Dance The Night" by Dua Lipa...
Done.
Searching for "vampire" by Olivia Rodrigo...
Done.
Searching for "Barbie World" by Nicki Minaj...
Done.


### Sentiment Analysis

In [15]:
top_five_uk_polarity = [get_polarity_score(song) for artist, song in top_five_uk]
print(top_five_uk_polarity)

[-0.04633838383838384, 0.09930555555555556, 0.29214285714285715, -0.0877513227513227, -0.17378787878787874]


## Top Songs Canada

### Get Track Names

In [16]:
top_five_canada = get_top_five('37i9dQZEVXbMda2apknTqH')
print(top_five_canada)

[('Travis Scott', 'MELTDOWN'), ('Gunna', 'fukumean'), ('Travis Scott', 'FE!N'), ('Morgan Wallen', 'Last Night'), ('Nicki Minaj', 'Barbie World')]


### Get Lyrics

In [17]:
for artist, song in top_five_canada:
    get_song_lyrics(song, artist)

Searching for "MELTDOWN" by Travis Scott...
Done.
Searching for "fukumean" by Gunna...
Done.
Searching for "FE!N" by Travis Scott...
Done.
Searching for "Last Night" by Morgan Wallen...
Done.
Searching for "Barbie World" by Nicki Minaj...
Done.


### Sentiment Analysis

In [18]:
top_five_canada_polarity = [get_polarity_score(song) for artist, song in top_five_canada]
print(top_five_canada_polarity)

[-0.16178026834276832, -0.000502953119232195, -0.08304473304473305, 0.05311224489795918, -0.17378787878787874]


## Top Songs Australia

### Get Track Names

In [19]:
top_five_australia = get_top_five('37i9dQZEVXbK4fwx2r07XW')
print(top_five_australia)

[('Dave', 'Sprinter'), ('Billie Eilish', 'What Was I Made For?'), ('Nicki Minaj', 'Barbie World'), ('Olivia Rodrigo', 'vampire'), ('Dua Lipa', 'Dance The Night')]


### Get Lyrics

In [20]:
for artist, song in top_five_australia:
    get_song_lyrics(song, artist)

Searching for "Sprinter" by Dave...
Done.
Searching for "What Was I Made For?" by Billie Eilish...
Done.
Searching for "Barbie World" by Nicki Minaj...
Done.
Searching for "vampire" by Olivia Rodrigo...
Done.
Searching for "Dance The Night" by Dua Lipa...
Done.


### Sentiment Analysis

In [21]:
top_five_australia_polarity = [get_polarity_score(song) for artist, song in top_five_australia]
print(top_five_australia_polarity)

[-0.04633838383838384, 0.09930555555555556, -0.17378787878787874, -0.0877513227513227, 0.29214285714285715]


## Playlist Comparison